In [1]:
%cd ../..

/home/ivanstefanov/Repositories/used-cars-price-prediction


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [5]:
from price_predictor.data_preparation.feature_engineering import add_class_feature
from price_predictor.data_preparation.pick_or_drop_features import (
    pick_by_datatypes,
    drop_anonymous_features,
    drop_categorical_features
)
from price_predictor.trainers.train_linear_model import train_linear_regression
from price_predictor.trainers.train_ensamble_model import train_random_forest, train_ada_boost
from price_predictor.evaluate import evaluate_model

In [6]:
data = pd.read_csv('data/cars.csv')
data.dropna(inplace=True)

## Try numerical with class of car

In [7]:
df = add_class_feature(data)

df = drop_anonymous_features(df)
df = drop_categorical_features(df)
df

,odometer_value,year_produced,engine_has_gas,engine_capacity,has_warranty,price_usd,is_exchangeable,number_of_photos,up_counter,duration_listed,class
0,190000,2010,False,2.5,False,10900.00,False,9,13,16,mid-end
1,290000,2002,False,3.0,False,5000.00,True,12,54,83,mid-end
2,402000,2001,False,2.5,False,2800.00,True,4,72,151,mid-end
3,10000,1999,False,3.0,False,9999.00,True,9,42,86,mid-end
4,280000,2001,False,2.5,False,2134.11,True,14,7,7,mid-end
...,...,...,...,...,...,...,...,...,...,...,...
38526,290000,2000,False,3.5,False,2750.00,True,5,85,301,mid-end
38527,321000,2004,False,2.2,False,4800.00,True,4,20,317,mid-end
38528,777957,2000,False,3.5,False,4300.00,False,3,63,369,mid-end
38529,20000,2001,False,2.0,False,4000.00,True,7,156,490,mid-end


In [14]:
dummies = pd.get_dummies(df)
scaled_dummies = pd.DataFrame(MinMaxScaler().fit_transform(dummies), index=dummies.index, columns=dummies.columns)
scaled_dummies

,odometer_value,year_produced,engine_has_gas,engine_capacity,has_warranty,price_usd,is_exchangeable,number_of_photos,up_counter,duration_listed,class_high-end,class_low-end,class_mid-end
0,0.190000,0.883117,0.0,0.294872,0.0,0.217984,0.0,0.094118,0.006452,0.007168,0.0,0.0,1.0
1,0.290000,0.779221,0.0,0.358974,0.0,0.099982,1.0,0.129412,0.028495,0.037186,0.0,0.0,1.0
2,0.402000,0.766234,0.0,0.294872,0.0,0.055981,1.0,0.035294,0.038172,0.067652,0.0,0.0,1.0
3,0.010000,0.740260,0.0,0.358974,0.0,0.199964,1.0,0.094118,0.022043,0.038530,0.0,0.0,1.0
4,0.280000,0.766234,0.0,0.294872,0.0,0.042663,1.0,0.152941,0.003226,0.003136,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
38526,0.290000,0.753247,0.0,0.423077,0.0,0.054981,1.0,0.047059,0.045161,0.134857,0.0,0.0,1.0
38527,0.321000,0.805195,0.0,0.256410,0.0,0.095982,1.0,0.035294,0.010215,0.142025,0.0,0.0,1.0
38528,0.777957,0.753247,0.0,0.423077,0.0,0.085982,0.0,0.023529,0.033333,0.165323,0.0,0.0,1.0
38529,0.020000,0.766234,0.0,0.230769,0.0,0.079982,1.0,0.070588,0.083333,0.219534,0.0,0.0,1.0


In [15]:
X_train, X_test, y_train, y_test = train_test_split(scaled_dummies.drop('price_usd', axis=1), scaled_dummies['price_usd'])

## Linear Regression

In [16]:
lr = train_linear_regression(X_train, y_train)
evaluate_model(lr, X_test, y_test)

{'rmse': 0.07500409592333032,
 'mae': 0.047601994219254276,
 'r2': 0.6612018466832905}

## Random Forest

In [18]:
rf = train_random_forest(X_train, y_train)
evaluate_model(rf, X_test, y_test)

{'rmse': 0.04619491308654512,
 'mae': 0.02866481845207747,
 'r2': 0.8714833542650766}

## Ada Boost

In [ ]:
ab = train_ada_boost(X_train, y_train)
evaluate_model(ab, X_test, y_test)

{'rmse': 4735.899505716994, 'mae': 4152.45161106216, 'r2': 0.4656799304936481}

## Conclusion: this car class feature doen't add useful information